## Monte Carlo Methods:  
* Definition:
** Using (pseudo)random numbers to solve (not-so-random) problems.
General approach
* Run Trials
** In each trial, simulate event (coin toss, dice rolling, etc)
* Count # of successful trials
** Guess that Expected Odds ~= Observed Odds = (successful trials) / (total trials)
* Law of Large Numbers
** As total trials increases, observed odds --> expected odds.
* Time-Accuracy Tradeoff
** More trials == more accurate + more time

## 1. Finding pi on a desert isle

Here, we have fun approximating pi by repeatedly throwing a coconut into a circle we inscribed in a square (using a vine we found on the beach). If the coconut throws are random, and we ignore throws that land outside the square, then the odds that a throw lands inside the circle are the ratio of the area of the circle divided by the area of the square, which equals pi/4. So we guess that pi is about 4 times our observed ratio.

In [12]:
import random, math

def randomFloatInRange(lo, hi):
    return lo + (hi - lo)*random.random()

def inUnitCircle(x,y):
    return (x**2 + y**2 <= 1)

def findPi(throws):
    throwsInCircle = 0
    for throw in range(throws):
        x = randomFloatInRange(-1, +1)
        y = randomFloatInRange(-1, +1)
        if inUnitCircle(x,y):
            throwsInCircle += 1
    return 4.0 * throwsInCircle / throws

for k in range(1,7):
    throws = 10**k
    piGuess = findPi(throws)
    accuracy = 100 * (1 - abs(piGuess - math.pi) / math.pi)
    print("10**%d throws --> %f (accuracy: %f%%)" %
          (k, piGuess, accuracy))
    

10**1 throws --> 3.200000 (accuracy: 98.140836%)
10**2 throws --> 3.000000 (accuracy: 95.492966%)
10**3 throws --> 3.132000 (accuracy: 99.694656%)
10**4 throws --> 3.135600 (accuracy: 99.809248%)
10**5 throws --> 3.135000 (accuracy: 99.790149%)
10**6 throws --> 3.141228 (accuracy: 99.988393%)


## 2. Random Run Length

We will say that a "run" of random numbers continues until you pick one that is smaller than the previous one. That last number ends the run, and is also counted in the run (so the shortest possible run length is 2). With this definition, what is the expected length of a run of random numbers?

In [ ]:
# Q: We will say that a "run" of random numbers continues
#    until you pick one that is smaller than the previous one.
#    That last number ends the run, and is also counted in the run
#    (so the shortest possible run length is 2).
#    With this definition, what is the expected length of
#    a run of random numbers?

# A: See code below.

import random

def randomRunLength():
    val = random.random()
    runLength = 1
    while True:
        nextVal = random.random()
        runLength += 1
        if (nextVal < val): 
            return runLength
        val = nextVal

# Or, if you prefer...
def randomRunLength():
    vals = [ random.random(), random.random() ]
    while (vals[-2] < vals[-1]): vals.append(random.random())
    return (len(vals))

def averageRandomRunLength(trials):
    totalRunLength = 0
    for trial in range(trials):
        totalRunLength += randomRunLength()
    return totalRunLength / trials

for k in range(1,6):
    trials = 10**k
    print("10**%d trials --> %f" %
          (k, averageRandomRunLength(trials)))

## Dice Throwing Tables

Here we compute the odds of getting various sums by rolling different numbers of different-sided dice (say, rolling 4 5-sided dice). There are many web resources to check your answer, such as this one (chosen randomly).

In [13]:
# For correct answers, try here:
# http://www.ogmiosproject.org/articles/stattables.html

import random

def rollDie(sides):
    return random.randint(1,sides)

def trialSucceeds(dice, sides, total):
    # run only ONE trial and return True on success.
    # one trial means:
    # roll 2 dice and succeed if their sum is total
    dieTotal = 0
    for roll in range(dice):
        die = rollDie(sides)
        dieTotal += die
    return (dieTotal == total)

def diceOdds(dice, sides, total, trials):
    successes = 0
    for trial in range(trials):
        if trialSucceeds(dice, sides, total) == True:
            successes += 1
    return successes / trials

def printAllDiceOdds(dice, sides):
    # print the label line
    print("Trials", end="")
    for label in range(dice, dice*sides+1):
        print("%7d" % label, end="")
    print()
    # now compute and print results for different-sized trials
    for k in range(1,5):
        trials = 10**k
        print("%7d" % trials, end="")
        for total in range(dice, dice*sides+1):
            print("%6.2f " % (100*diceOdds(dice, sides, total, trials)),
                  end="")
        print()

printAllDiceOdds(3,6)

Trials      3      4      5      6      7      8      9     10     11     12     13     14     15     16     17     18
     10  0.00   0.00   0.00  10.00  10.00  20.00  50.00  20.00  20.00  10.00   0.00  20.00  10.00   0.00   0.00   0.00 
    100  0.00   3.00   1.00   7.00   7.00   7.00  12.00  16.00  11.00  11.00  11.00   8.00   7.00   1.00   3.00   0.00 
   1000  0.60   1.20   3.50   4.30   7.20   9.70  10.90  10.80  12.50  10.50  10.60   6.60   4.80   3.30   1.80   0.30 
  10000  0.45   1.21   3.01   4.60   7.29   9.81  11.65  12.17  12.72  11.22   9.89   6.80   4.70   2.61   1.47   0.50 


In [1]:
# Here we improve on diceThrowing.py by realizing that we can
# compute the odds for all the totals at once, rather than
# repeating our trials for each possible total.

import random

def rollDie(sides):
    return random.randint(1,sides)

def rollDice(dice, sides):
    dieTotal = 0
    for roll in range(dice):
        die = rollDie(sides)
        dieTotal += die
    return dieTotal

def diceOdds(dice, sides):
    trials = 1000*100
    maxTotal = dice*sides
    successes = [0]*(maxTotal+1)
    for trial in range(trials):
        dieTotal = rollDice(dice, sides)
        successes[dieTotal] += 1
    odds = [count/trials for count in successes]
    return odds

def printAllDiceOdds(dice, sides):
    odds = diceOdds(dice, sides)
    for total in range(dice,dice*sides+1):
        print(total, "%6.2f" % (100*odds[total]))

printAllDiceOdds(3,6)

3   0.48
4   1.37
5   2.85
6   4.63
7   7.06
8   9.57
9  11.45
10  12.45
11  12.53
12  11.43
13   9.77
14   7.01
15   4.66
16   2.81
17   1.41
18   0.50


## The Birthday Problem

And here we solve the famous birthday problem: how many people must be in a room so that it is more likely than not that at least two of them share a birthday (same day and month, not necessarily year; and we ignored leap year birthdays)? You can check your answer at the Wikipedia page on the Birthday Problem.

In [14]:
import random

def trialSucceeds(numberOfPeopleInRoom):
    # run only ONE trial and return True on success.
    # one trial means:
    # Put the given # of people in a room
    # and return true if any 2 share any birthday
    seenBirthdays = set()
    for person in range(numberOfPeopleInRoom):
        birthday = random.randint(1,365)
        if (birthday in seenBirthdays):
            return True
        seenBirthdays.add(birthday)
    return False

def birthdayOdds(numberOfPeopleInRoom, trials):
    successes = 0
    for trial in range(trials):
        if trialSucceeds(numberOfPeopleInRoom) == True:
            successes += 1
    return successes / trials

def solveBirthdayProblem(trials):
    for peopleInRoom in range(2,366):
        if (birthdayOdds(peopleInRoom, trials) >= 0.5):
            return peopleInRoom

for k in range(1,6):
    trials = 10**k
    print("10**%d trials --> %f" %
          (k, solveBirthdayProblem(trials)))

10**1 trials --> 19.000000
10**2 trials --> 24.000000
10**3 trials --> 23.000000
10**4 trials --> 24.000000
10**5 trials --> 23.000000
